# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
#load environment variables
%load_ext dotenv
%dotenv ../01_materials/labs/.env

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\prasa\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

pd_dir = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(pd_dir, '**/*.parquet/*.parquet'))
pd_df = dd.read_parquet(parquet_files).compute().reset_index()

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
#converting pandas dataframe to dask dataframe
dd_df = dd.from_pandas(pd_df, npartitions = len('ticker'))

In [7]:
dd_feat = (dd_df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj Close'].shift(1)
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High']-x['Low'])
))

C:\Users\prasa\AppData\Local\Temp\ipykernel_20156\1979108503.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
pd_return = dd_feat.compute()
print(type(pd_return))

<class 'pandas.core.frame.DataFrame'>


In [10]:
pd_return['rolling_avg_10'] = pd_return.groupby('ticker')['returns'].rolling(10).mean().reset_index(0,drop=True)
pd_return

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling_avg_10
1862682,NVR,2000-01-03,47.750000,47.750000,46.500000,46.562500,46.562500,36700,Consumer Discretionary,Homebuilding,2000,NaN,NaN,NaN,1.250000,NaN
1862683,NVR,2000-01-04,46.375000,46.375000,46.000000,46.000000,46.000000,34100,Consumer Discretionary,Homebuilding,2000,46.562500,46.562500,-0.012081,0.375000,NaN
1862684,NVR,2000-01-05,46.000000,46.125000,45.750000,45.875000,45.875000,16500,Consumer Discretionary,Homebuilding,2000,46.000000,46.000000,-0.002717,0.375000,NaN
1862685,NVR,2000-01-06,45.625000,45.875000,45.500000,45.875000,45.875000,12700,Consumer Discretionary,Homebuilding,2000,45.875000,45.875000,0.000000,0.375000,NaN
1862686,NVR,2000-01-07,45.500000,46.750000,45.500000,46.750000,46.750000,73400,Consumer Discretionary,Homebuilding,2000,45.875000,45.875000,0.019074,1.250000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766586,MTD,2024-06-21,1458.180054,1469.640015,1453.030029,1463.479980,1463.479980,165900,Health Care,Life Sciences Tools & Services,2024,1456.589966,1456.589966,0.004730,16.609985,0.001819
1766587,MTD,2024-06-24,1468.510010,1468.510010,1446.530029,1453.979980,1453.979980,105700,Health Care,Life Sciences Tools & Services,2024,1463.479980,1463.479980,-0.006491,21.979980,0.001132
1766588,MTD,2024-06-25,1449.420044,1451.069946,1415.819946,1426.800049,1426.800049,115700,Health Care,Life Sciences Tools & Services,2024,1453.979980,1453.979980,-0.018693,35.250000,-0.001102
1766589,MTD,2024-06-26,1426.800049,1429.280029,1409.650024,1427.300049,1427.300049,108100,Health Care,Life Sciences Tools & Services,2024,1426.800049,1426.800049,0.000350,19.630005,-0.000899


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
'''No it is not necesary to convert to pandas to calculate the moving average.'''

''' In pandas calculating moving average is straigntforward as it has a built in rolling window function.So if the dataset 
is small pandas is more efficient.
For large datasets where parallel processing is required using Dask could be a better choice but a custom function is required
to calculate moving average'''

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.